In [2]:
%pip install groq python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 2.2 MB/s eta 0:00:00


In [9]:
import os
import getpass
from dotenv import load_dotenv
from groq import Groq

# =========================================================
# 1. SETUP (SECURE API KEY INPUT USING GETPASS)
# =========================================================

load_dotenv()

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

client = Groq(api_key=os.environ["GROQ_API_KEY"])

BASE_MODEL = "llama-3.1-8b-instant"# =========================================================
# 2. DEFINE EXPERT CONFIGURATIONS
# =========================================================

MODEL_CONFIG = {
    "technical": {
        "system_prompt": (
            "You are a senior technical support engineer. "
            "Your responses must be precise, code-focused, and technically accurate. "
            "Provide debugging steps and example fixes when possible."
        ),
        "temperature": 0.7,
    },
    "billing": {
        "system_prompt": (
            "You are a customer billing specialist. "
            "You are empathetic, professional, and policy-driven. "
            "Explain billing issues clearly and outline refund or escalation steps."
        ),
        "temperature": 0.7,
    },
    "general": {
        "system_prompt": (
            "You are a friendly customer support assistant. "
            "Handle general questions and casual conversation clearly and politely."
        ),
        "temperature": 0.7,
    },
}

# =========================================================
# 3. ROUTER FUNCTION
# =========================================================

def route_prompt(user_input: str) -> str:
    """
    Classifies the user's query into one of the predefined categories.

    Parameters:
        user_input (str): The user's input text.

    Returns:
        str: One of ['technical', 'billing', 'general'].
    """
    routing_prompt = (
        "Classify the following user query into one of these categories:\n"
        "technical, billing, general.\n\n"
        "Return ONLY the category name.\n\n"
        f"User Query: {user_input}"
    )

    response = client.chat.completions.create(
        model=BASE_MODEL,
        messages=[
            {"role": "system", "content": "You are an intent classification engine."},
            {"role": "user", "content": routing_prompt},
        ],
        temperature=0.0,
        max_tokens=10,
    )

    return response.choices[0].message.content.strip().lower()

# =========================================================
# 4. ORCHESTRATOR
# =========================================================

def process_request(user_input: str) -> str:
    """
    Routes the user input to the appropriate expert and returns the expert response.

    Parameters:
        user_input (str): The user's query.

    Returns:
        str: The expert-generated response.
    """
    category = route_prompt(user_input)

    expert_config = MODEL_CONFIG.get(category, MODEL_CONFIG["general"])

    response = client.chat.completions.create(
        model=BASE_MODEL,
        messages=[
            {"role": "system", "content": expert_config["system_prompt"]},
            {"role": "user", "content": user_input},
        ],
        temperature=expert_config["temperature"],
        max_tokens=500,
    )

    return response.choices[0].message.content

# =========================================================
# 5. EXAMPLE USAGE
# =========================================================

if __name__ == "__main__":
    queries = [
        "My python script is throwing an IndexError on line 5.",
        "I was charged twice for my subscription this month.",
        "Hello, how does your service work?",
    ]

    for q in queries:
        print(f"\nUser: {q}")
        print(f"Assistant: {process_request(q)}")


User: My python script is throwing an IndexError on line 5.
Assistant: To troubleshoot the `IndexError`, I'll need more information about your script. However, I can guide you through a general approach to resolve the issue.

**Step 1: Identify the relevant code snippet**

Please provide the code snippet where the `IndexError` occurs, especially the line where the error happens (line 5).

**Step 2: Check the length of the list**

An `IndexError` usually occurs when you're trying to access an element at an index that's out of range. Make sure the list or array you're accessing has at least as many elements as the index you're trying to access.

**Step 3: Review the indexing**

Double-check the indexing. Python uses 0-based indexing, meaning the first element is at index 0, not 1. Ensure you're not trying to access an element at an index that's greater than or equal to the length of the list.

**Step 4: Provide the code snippet**

Please paste the relevant code snippet (e.g., 5-10 lines